## 환경 설정
- 공유 드라이브에서 ipynb 파일 다운로드
- colab.research.google.com 접속
- 파일 > 노트 업로드 > 다운 받은 ipynb 파일 업로드
- 수정 > 노트 설정 > 하드웨어 가속기 GPU 여부 확인

## Huggingface
- 각종 딥러닝 모델 구현을 위한 transformers 라이브러리
  - https://huggingface.co/docs/transformers/index
- 공개 모델 및 데이터셋 플랫폼 제공
  - https://huggingface.co/models
  - https://huggingface.co/datasets

## 실습 1-1. PyTorch를 이용한 Fine-tuning
- PyTorch 기본 기능을 이용해서 리뷰 분류 모델을 학습
- yelp 데이터셋 ("yelp_review_full") : 식당 리뷰 데이터. 리뷰 문장과 긍정, 부정 레이블로 구성

In [ ]:
# 필요한 라이브러리 설치
!pip install transformers datasets evaluate -q

In [ ]:
import torch
import evaluate

from tqdm.auto import tqdm
from torch.optim import AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler

In [ ]:
# yelp review 데이터셋 사용
# train set 1,000개. test set 100개 샘플링
dataset = load_dataset("yelp_review_full")
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(100))

In [ ]:
small_train_dataset[100]

In [ ]:
# 데이터셋 로컬에 저장하기
small_eval_dataset.to_json("data.json")

In [ ]:
!head data.json

In [ ]:
# 로컬 파일을 데이터셋으로 불러오기
sample_dataset_for_loading = load_dataset("json", data_files={"test": "data.json"})
sample_dataset_for_loading

In [ ]:
# bert tokenizer 설정
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# tokenization
tokenized_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

In [ ]:
type(tokenized_train_dataset), len(tokenized_train_dataset)

In [ ]:
type(tokenized_train_dataset[0])

In [ ]:
tokenized_train_dataset[0].keys()

In [ ]:
for k, v in tokenized_train_dataset[0].items():
    print(k, v)

In [ ]:
# tokenized dataset을 torch에서 사용 가능한 포맷으로 처리
train_tensor = tokenized_train_dataset.remove_columns(["text"])
train_tensor = train_tensor.rename_column("label", "labels")
train_tensor.set_format("torch")

eval_tensor = tokenized_eval_dataset.remove_columns(["text"])
eval_tensor = eval_tensor.rename_column("label", "labels")
eval_tensor.set_format("torch")

In [ ]:
train_tensor[0].keys()

In [ ]:
for k, v in train_tensor[0].items():
    if k == "labels":
        print(k, v)
    else:
        print(k, v[:200])

In [ ]:
# dataloader, model, optimizer 설정
train_dataloader = DataLoader(train_tensor, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(eval_tensor, batch_size=8)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
# scheduler 설정
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
# model device 설정
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
# 모델 성능 평가
def eval_model():
    metric = evaluate.load("accuracy")
    model.eval()

    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

    return metric.compute()

In [ ]:
# 학습 전 성능 확인
eval_model()

In [ ]:
# 모델 학습
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
# 학습 후 성능 확인
eval_model()

In [ ]:
# 모델 output 확인용 샘플
raw = tokenized_eval_dataset[0]
data = eval_tensor[0]

for k, v in raw.items():
    print(k, v)

print()

for k, v in data.items():
    if k == "labels":
        print(k, v)
    else:
        print(k, v[:200])

In [ ]:
outputs = model(
    input_ids=data["input_ids"].view(1, -1).to(device),
    token_type_ids=data["token_type_ids"].view(1, -1).to(device),
    attention_mask=data["attention_mask"].view(1, -1).to(device)
)
outputs.logits.softmax(dim=1)

## 실습 1-2. 네이버 영화 리뷰 Fine-tuning
- 이전 내용을 참고해서 영화 리뷰 분류 모델을 학습시켜 봅시다.
  - 데이터셋 및 모델 설정 -> 학습 전 성능 평가 -> 학습 -> 학습 후 성능 평가
- 네이버 영화 리뷰 데이터셋 사용 ("nsmc")
  - 길이 20자 이상 데이터만 사용
  - Train Set 3,000개 샘플링 (seed=42)
  - Test Set 100개 샘플링 (seed=42)
  - 길이 필터링 -> 셔플링 -> 샘플링
- 1 Epoch 학습
- 20 step 단위로 누적 평균 loss를 구해봅시다. (loss sum / step count)
- 한글 데이터를 처리하기 위해 SKT KoBERT를 사용합니다. ("skt/kobert-base-v1")
  - https://github.com/SKTBrain/KoBERT

In [ ]:
# 필요한 라이브러리 설치
!pip install transformers datasets evaluate sentencepiece -q
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf' -q

In [ ]:
# KoBERT Tokenizer 라이브러리 사용
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')